# Домашнее задание 

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

Cumulative gain at K (CG@K) - базовая метрика ранжирования, которая использует простую идею: чем релевантные элементы в этом топе, тем лучше. Эта метрика обладает очевидными недостатками: она не нормализована и не учитывает позицию релевантных элементов. 

Discounted cumulative gain at K (DCG@K) - модификация cumulative gain at K, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции

PFound - метрика качества ранжирования, использующая похожую на каскадную модель

Fraction of Concordance Pairs - насколько высока концентрация интересных товаров в начале списка рекомендаций.

**2) Доделать все функции, где стоит комментарий "сделать дома"**

In [1]:
import pandas as pd
import numpy as np

In [2]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] 
bought_list = [521, 32, 143, 991]

prices_recommended = [400, 60, 40, 40 , 90]
prices_bought = [300, 110, 90, 40, 90]

In [3]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1
    hit_rate_at_k = (flags.sum() > 0) * 1
    return hit_rate_at_k

In [4]:
hit_rate_at_k(recommended_list, bought_list, k=5)

1

In [5]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])

    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])
    prices_bought = np.array(prices_bought)

    flags = np.isin(recommended_list, bought_list)

    return (prices_recommended * flags).sum() / prices_bought.sum()

In [6]:
recall_at_k(recommended_list, bought_list, k=5)

0.5

In [7]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.6984126984126984

In [8]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

def money_precision_at_k_(recommended_list, bought_list, prices_recommended, k=5):
    
    recommend_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [9]:
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                            [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                            [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]
                      ] # юзер 3

In [10]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    for index_relevant in relevant_indexes:
        print(precision_at_k(recommended_list, bought_list, k=index_relevant + 1) )
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant + 1) for index_relevant in relevant_indexes])
    
    return sum_/amount_relevant

def map_k(recommended_list_users, bought_list_users, k=5):    
    user_qty = len(recommended_list_users)
    ap_k_sum = sum([ap_k(recommended_list_users[i], bought_list_users[i], k) for i in range(0,user_qty)])
    return (1/user_qty)*ap_k_sum

In [11]:
map_k(recommended_list_3_users, bought_list_3_users, k=5)

1.0


0.3333333333333333

In [12]:
def reciprocal_rank(recommended_list_users, bought_list_users, k=1): 
    rs = []
    users_qty = len(recommended_list_users)
    
    i = 0
    while i < users_qty:
        bought_list = np.array(bought_list_users[i])
        recommended_list = np.array(recommended_list_users[i] [:k])
        
        rs.append(np.nonzero(np.isin(recommended_list, bought_list))[0])
        i = i+1
        
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

In [13]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=1)

0.3333333333333333